# 판매 채널 데이터 전처리: Wide → Long 변환 (영문 버전)

**목표:** 연도별로 분산된 판매 데이터를 하나의 테이블로 통합

## Step 1: 라이브러리 import & 파일 읽기

In [101]:
import pandas as pd

# 파일 읽기 (실제 파일 경로로 수정)
df = pd.read_excel(r'C:\Users\Comet\Desktop\kpdh\data\raw\museum\csv\museum_shop_sales.xlsx',sheet_name=1)

# 원본 확인
print("원본 데이터:")
display(df)
print(f"\n컬럼: {df.columns.tolist()}")

# 공백 제거 (안전장치)
df['구분'] = df['구분'].astype(str).str.strip()


원본 데이터:


,구분,2021년,2021년_구성비,2022년,2022년_구성비,2023년,2023년_구성비,2024년,2024년_구성비,2025년,2025년_구성비
0,계,"6,591",100.0%,"11,692",100.0%,"14,976",100.0%,"21,284",100.0%,"41,337",100.0%
1,오프라인,"3,259",49.4%,"7,349",62.9%,"9,477",63.3%,"11,756",55.2%,"23,391",56.6%
2,온라인,"1,629",24.7%,"1,931",16.5%,"2,229",14.9%,"5,222",24.5%,"12,187",29.5%
3,기업특판(B2B),"1,702",25.8%,"2,413",20.6%,"3,171",21.2%,"4,024",18.9%,"5,623",13.6%
4,기타(로열티),0,0.0%,0,0.0%,98,0.7%,282,1.3%,135,0.3%



컬럼: ['구분', '2021년', '2021년_구성비', '2022년', '2022년_구성비', '2023년', '2023년_구성비', '2024년', '2024년_구성비', '2025년', '2025년_구성비']


## Step 2: '계' 행 제거

In [102]:
# 현재 컬럼명 확인
print("현재 컬럼 목록:")
print(df.columns.tolist())

# 데이터 미리보기
print("\n데이터 미리보기:")
display(df.head())

# '계' 행 확인
print(f"\n'계' 행 개수: {(df['구분'] == '계').sum()}개")


현재 컬럼 목록:
['구분', '2021년', '2021년_구성비', '2022년', '2022년_구성비', '2023년', '2023년_구성비', '2024년', '2024년_구성비', '2025년', '2025년_구성비']

데이터 미리보기:


,구분,2021년,2021년_구성비,2022년,2022년_구성비,2023년,2023년_구성비,2024년,2024년_구성비,2025년,2025년_구성비
0,계,"6,591",100.0%,"11,692",100.0%,"14,976",100.0%,"21,284",100.0%,"41,337",100.0%
1,오프라인,"3,259",49.4%,"7,349",62.9%,"9,477",63.3%,"11,756",55.2%,"23,391",56.6%
2,온라인,"1,629",24.7%,"1,931",16.5%,"2,229",14.9%,"5,222",24.5%,"12,187",29.5%
3,기업특판(B2B),"1,702",25.8%,"2,413",20.6%,"3,171",21.2%,"4,024",18.9%,"5,623",13.6%
4,기타(로열티),0,0.0%,0,0.0%,98,0.7%,282,1.3%,135,0.3%



'계' 행 개수: 1개


In [103]:
# '계' 행 제거
df = df[df['구분'] != '계'].copy()

print(f"\n'계' 제거 후 shape: {df.shape}")
print(f"남은 구분: {df['구분'].unique()}")

# 데이터 확인
display(df)


'계' 제거 후 shape: (4, 11)
남은 구분: ['오프라인' '온라인' '기업특판(B2B)' '기타(로열티)']


,구분,2021년,2021년_구성비,2022년,2022년_구성비,2023년,2023년_구성비,2024년,2024년_구성비,2025년,2025년_구성비
1,오프라인,"3,259",49.4%,"7,349",62.9%,"9,477",63.3%,"11,756",55.2%,"23,391",56.6%
2,온라인,"1,629",24.7%,"1,931",16.5%,"2,229",14.9%,"5,222",24.5%,"12,187",29.5%
3,기업특판(B2B),"1,702",25.8%,"2,413",20.6%,"3,171",21.2%,"4,024",18.9%,"5,623",13.6%
4,기타(로열티),0,0.0%,0,0.0%,98,0.7%,282,1.3%,135,0.3%


## Step 3: 2021년 데이터만 변환 테스트

In [104]:
# 2021년 데이터 변환 테스트
if len(df) > 0:
    temp_2021 = df[['구분', '2021년', '2021년_구성비']].copy()
    temp_2021.columns = ['channel', 'sales', 'ratio']
    temp_2021['year'] = 2021
    
    print("\n2021년 데이터 변환 결과:")
    display(temp_2021)
else:
    print("\n⚠️ 데이터가 없습니다!")


2021년 데이터 변환 결과:


,channel,sales,ratio,year
1,오프라인,"3,259",49.4%,2021
2,온라인,"1,629",24.7%,2021
3,기업특판(B2B),"1,702",25.8%,2021
4,기타(로열티),0,0.0%,2021


## Step 4: 모든 연도 데이터 변환 & 합치기

In [105]:
# 11단계: ratio 직접 계산 (소수 형태, 소수점 4자리)
df_long['ratio'] = df_long.groupby('year')['sales'].transform(
    lambda x: (x / x.sum()).round(4)
)

# 12단계: 반올림 오차 보정 (가장 큰 값에 조정)
for year in df_long['year'].unique():
    year_mask = df_long['year'] == year
    total = df_long.loc[year_mask, 'ratio'].sum()
    diff = 1.00 - total
    
    if diff != 0:
        # 마지막 행 대신 가장 큰 ratio에 조정
        max_idx = df_long[year_mask]['ratio'].idxmax()
        df_long.loc[max_idx, 'ratio'] += diff

# 13단계: 정렬
df_long = df_long.sort_values(['year', 'channel']).reset_index(drop=True)

# 14단계: 단위 컬럼 추가
df_long['unit'] = '백만원'

# 15단계: 채널명 영문 추가 (한글명도 유지)
channel_map = {
    '오프라인': 'offline',
    '온라인': 'online',
    '기업특판(B2B)': 'b2b',
    '기타(로열티)': 'royalty'
}

df_long['channel_kr'] = df_long['channel']  # 한글명
df_long['channel_en'] = df_long['channel'].map(channel_map)  # 영문명

# 16단계: 컬럼 순서 정리
df_long = df_long[['year', 'channel_kr', 'channel_en', 'sales', 'unit', 'ratio']]

# 17단계: 최종 결과 확인
print("\n최종 결과:")
display(df_long.head(15))

# 18단계: 검증
print("\n연도별 ratio 합계:")
print(df_long.groupby('year')['ratio'].sum())

# 19단계: 데이터 타입 확인
print("\n데이터 타입:")
print(df_long.dtypes)


최종 결과:


,year,channel_kr,channel_en,sales,unit,ratio
0,2021,b2b,NaN,1702,백만원,0.2583
1,2021,offline,NaN,3259,백만원,0.4945
2,2021,online,NaN,1629,백만원,0.2472
3,2021,royalty,NaN,0,백만원,0.0000
4,2022,b2b,NaN,2413,백만원,0.2064
5,2022,offline,NaN,7349,백만원,0.6285
6,2022,online,NaN,1931,백만원,0.1651
7,2022,royalty,NaN,0,백만원,0.0000
8,2023,b2b,NaN,3171,백만원,0.2118
9,2023,offline,NaN,9477,백만원,0.6329



연도별 ratio 합계:
year
2021    1.0
2022    1.0
2023    1.0
2024    1.0
2025    1.0
Name: ratio, dtype: float64

데이터 타입:
year            int64
channel_kr     object
channel_en     object
sales           int64
unit           object
ratio         float64
dtype: object


## Step 5: 컬럼 순서 정리

In [89]:
# 음수 ratio를 0으로 변경
df_long.loc[df_long['ratio'] < 0, 'ratio'] = 0.00

# 정렬
df_long = df_long.sort_values(['year', 'channel']).reset_index(drop=True)

# 채널명 영문 추가 (한글명도 유지)
channel_map = {
    '오프라인': 'offline',
    '온라인': 'online',
    '기업특판(B2B)': 'b2b',
    '기타(로열티)': 'royalty'
}

df_long['channel_kr'] = df_long['channel']  # 한글명
df_long['channel_en'] = df_long['channel'].map(channel_map)  # 영문명

# 컬럼 순서 정리
df_long = df_long[['year', 'channel_kr', 'channel_en', 'sales', 'unit', 'ratio']]

print("최종 결과:")
display(df_long.head(15))

# 검증
print("\n연도별 ratio 합계:")
print(df_long.groupby('year')['ratio'].sum())

# 데이터 타입
print("\n데이터 타입:")
print(df_long.dtypes)

# # CSV 저장
# df_long.to_csv('판매채널_전처리_완료.csv', index=False, encoding='utf-8-sig')
# print("\n✅ CSV 저장 완료!")

KeyError: "['unit'] not in index"

In [45]:
# 컬럼 순서 정리 (year를 맨 앞으로)
df = df[['year', 'channel', 'sales', 'ratio']]

print("컬럼 순서 정리 후:")
display(df.head(10))
print(f"\n컬럼: {df.columns.tolist()}")

컬럼 순서 정리 후:


,year,channel,sales,ratio
0,2021,오프라인,"3,259",49.4%
1,2021,온라인,"1,629",24.7%
2,2021,기업특판(B2B),"1,702",25.8%
3,2021,기타(로열티),0,0.0%
4,2022,오프라인,"7,349",62.9%
5,2022,온라인,"1,931",16.5%
6,2022,기업특판(B2B),"2,413",20.6%
7,2022,기타(로열티),0,0.0%
8,2023,오프라인,"9,477",63.3%
9,2023,온라인,"2,229",14.9%



컬럼: ['year', 'channel', 'sales', 'ratio']


## Step 6: 데이터 타입 변환

In [ ]:
# 변환 전 데이터 타입 확인
print("변환 전 데이터 타입:")
print(df.dtypes)
print("\n샘플 데이터:")
display(df.head(3))

변환 전 데이터 타입:
year        int64
channel    object
sales      object
ratio      object
dtype: object

샘플 데이터:


,year,channel,sales,ratio
0,2021,오프라인,"3,259",49.4%
1,2021,온라인,"1,629",24.7%
2,2021,기업특판(B2B),"1,702",25.8%


In [50]:
# 금액: 콤마 제거 후 정수로 변환
df['sales'] = df['sales'].astype(str).str.replace(',', '').astype(int)

# 구성비: % 제거 후 실수로 변환
df['ratio'] = df['ratio'].astype(str).str.replace('%', '').astype(float)

print("\n변환 후 데이터 타입:")
print(df.dtypes)
print("\n변환 결과:")
display(df.head(5))


변환 후 데이터 타입:
year         int64
channel     object
sales        int64
ratio      float64
dtype: object

변환 결과:


,year,channel,sales,ratio
0,2021,오프라인,3259,49.4
1,2021,온라인,1629,24.7
2,2021,기업특판(B2B),1702,25.8
3,2021,기타(로열티),0,0.0
4,2022,오프라인,7349,62.9


## Step 7: 채널명 영문으로 변경

In [ ]:
# 채널명 매핑 딕셔너리
channel_map = {
    '오프라인': 'offline',
    '온라인': 'online',
    '기업특판(B2B)': 'b2b',
    '기타(로열티)': 'royalty'
}

# 변경 전
print("변경 전 채널명:")
print(df_long['channel'].unique())

# 영문으로 변경
df_long['channel'] = df_long['channel'].map(channel_map)

# 변경 후
print("\n변경 후 채널명:")
print(df_long['channel'].unique())
print("\n변경 결과:")
display(df_long.head(10))

## Step 8: 정렬

In [ ]:
# 연도 → 채널 순으로 정렬
df_long = df_long.sort_values(['year', 'channel']).reset_index(drop=True)

print("정렬 후 최종 결과:")
display(df_long.head(15))
print(f"\n최종 shape: {df_long.shape}")
print(f"컬럼: {df_long.columns.tolist()}")

## Step 9: 기본 통계 및 확인

In [ ]:
# 결측치 확인
print("결측치 개수:")
print(df_long.isnull().sum())

# 기본 통계
print("\n금액 기본 통계:")
display(df_long['amount'].describe())

# 연도별 총 금액
print("\n연도별 총 금액:")
year_sum = df_long.groupby('year')['amount'].sum().sort_index()
display(year_sum)

# 채널별 총 금액
print("\n채널별 총 금액 (2021-2025 합계):")
channel_sum = df_long.groupby('channel')['amount'].sum().sort_values(ascending=False)
display(channel_sum)

## Step 10: 피벗 테이블로 확인

In [ ]:
# 연도/채널별 피벗 테이블
print("연도/채널별 금액 피벗 테이블:")
pivot_amount = df_long.pivot_table(
    values='amount',
    index='channel',
    columns='year',
    aggfunc='sum'
)
display(pivot_amount)

print("\n연도/채널별 구성비 피벗 테이블:")
pivot_ratio = df_long.pivot_table(
    values='ratio',
    index='channel',
    columns='year',
    aggfunc='mean'
)
display(pivot_ratio)

## Step 11: CSV 파일로 저장

In [ ]:
# CSV 파일로 저장
df_long.to_csv('판매채널_전처리_완료.csv', index=False, encoding='utf-8-sig')

print("✅ CSV 저장 완료: 판매채널_전처리_완료.csv")
print(f"저장된 데이터 수: {len(df_long)}행")

---

## 완료! 전처리 요약

**변환 내용:**
1. Wide 형태 → Long 형태 변환
2. 5개 연도 데이터 통합 (2021-2025)
3. 컬럼명 영문 변경 (year, channel, amount, ratio)
4. 채널명 영문 변경 (offline, online, b2b, royalty)
5. 데이터 타입 변환 (콤마/% 제거)

**최종 컬럼:**
- year: 연도
- channel: 판매 채널
- amount: 금액 (원)
- ratio: 구성비 (%)

**다음 단계:**
- 시계열 분석
- 채널별 성장률 계산
- Tableau/Power BI 시각화

In [111]:
import pandas as pd

# ========================================
# Step 1: 파일 읽기
# ========================================
# 파일 읽기 
df = pd.read_excel(r'C:\Users\Comet\Desktop\kpdh\data\raw\museum\csv\museum_shop_sales.xlsx',sheet_name=1)

# 원본 확인
print("원본 데이터:")
display(df)
print(f"\n원본 shape: {df.shape}")
print(f"\n컬럼: {df.columns.tolist()}")

# ========================================
# Step 2: '계' 행 제거
# ========================================
# 공백 제거 (안전장치)
df['구분'] = df['구분'].astype(str).str.strip()

# '계' 행 확인
print(f"\n'계' 행 개수: {(df['구분'] == '계').sum()}개")

# '계' 행 제거
df = df[df['구분'] != '계'].copy()

print(f"\n'계' 제거 후 shape: {df.shape}")
print(f"남은 구분: {df['구분'].unique()}")

# ========================================
# Step 3-7: 연도별 데이터 처리
# ========================================
# 빈 리스트 생성
dfs = []

# 2021년부터 2025년까지 반복 처리
for year in [2021, 2022, 2023, 2024, 2025]:
    # 현재 연도의 데이터만 추출 (구성비 컬럼 제외)
    temp = df[['구분', f'{year}년']].copy()
    
    # 컬럼명 설정 (한글명 유지)
    temp.columns = ['channel_kr', 'sales']
    
    # 연도 컬럼 추가
    temp['year'] = year
    
    # 리스트에 추가
    dfs.append(temp)
    print(f"{year}년 데이터 추가: {temp.shape}")

# ========================================
# Step 8: 모든 연도 합치기
# ========================================
df_long = pd.concat(dfs, ignore_index=True)

print("\n합친 결과:")
display(df_long.head(10))
print(f"\n전체 shape: {df_long.shape}")

# ========================================
# Step 9: 컬럼 순서 정리
# ========================================
df_long = df_long[['year', 'channel_kr', 'sales']]

# ========================================
# Step 10: sales 데이터 타입 변환
# ========================================
df_long['sales'] = df_long['sales'].astype(str).str.replace(',', '').astype(int)

print("\nsales 변환 후:")
display(df_long.head(5))

# ========================================
# Step 11: ratio 직접 계산
# ========================================
df_long['ratio'] = df_long.groupby('year')['sales'].transform(
    lambda x: (x / x.sum()).round(4)
)

# ========================================
# Step 12: 반올림 오차 보정
# ========================================
for year in df_long['year'].unique():
    year_mask = df_long['year'] == year
    total = df_long.loc[year_mask, 'ratio'].sum()
    diff = 1.00 - total
    
    if diff != 0:
        # 가장 큰 ratio에 조정
        max_idx = df_long[year_mask]['ratio'].idxmax()
        df_long.loc[max_idx, 'ratio'] += diff
        print(f"{year}년 보정: {diff:+.4f}")

# ========================================
# Step 13: 정렬
# ========================================
df_long = df_long.sort_values(['year', 'channel_kr']).reset_index(drop=True)

# ========================================
# Step 14: 채널명 영문 추가
# ========================================
channel_map = {
    '오프라인': 'Offline',
    '온라인': 'Online',
    '기업특판(B2B)': 'B2B',
    '기타(로열티)': 'Royalty'
}

# 영문명 생성 (한글 → 영문)
df_long['channel'] = df_long['channel_kr'].map(channel_map)

# ========================================
# Step 16: 컬럼 순서 정리
# ========================================
df_long = df_long[['year','channel','channel_kr', 'sales', 'ratio']]

# ========================================
# Step 17: 최종 결과 확인
# ========================================
print("\n" + "="*50)
print("최종 결과")
print("="*50)
display(df_long)

# ========================================
# Step 18: 검증
# ========================================
print("\n연도별 ratio 합계:")
ratio_check = df_long.groupby('year')['ratio'].sum()
display(ratio_check)

# ========================================
# Step 19: 데이터 타입 확인
# ========================================
print("\n데이터 타입:")
print(df_long.dtypes)

# ========================================
# Step 20: 기본 통계
# ========================================
print("\n연도별 총 매출 (백만원):")
display(df_long.groupby('year')['sales'].sum())

print("\n채널별 총 매출 (백만원):")
display(df_long.groupby('channel_kr')['sales'].sum().sort_values(ascending=False))


원본 데이터:


,구분,2021년,2021년_구성비,2022년,2022년_구성비,2023년,2023년_구성비,2024년,2024년_구성비,2025년,2025년_구성비
0,계,"6,591",100.0%,"11,692",100.0%,"14,976",100.0%,"21,284",100.0%,"41,337",100.0%
1,오프라인,"3,259",49.4%,"7,349",62.9%,"9,477",63.3%,"11,756",55.2%,"23,391",56.6%
2,온라인,"1,629",24.7%,"1,931",16.5%,"2,229",14.9%,"5,222",24.5%,"12,187",29.5%
3,기업특판(B2B),"1,702",25.8%,"2,413",20.6%,"3,171",21.2%,"4,024",18.9%,"5,623",13.6%
4,기타(로열티),0,0.0%,0,0.0%,98,0.7%,282,1.3%,135,0.3%



원본 shape: (5, 11)

컬럼: ['구분', '2021년', '2021년_구성비', '2022년', '2022년_구성비', '2023년', '2023년_구성비', '2024년', '2024년_구성비', '2025년', '2025년_구성비']

'계' 행 개수: 1개

'계' 제거 후 shape: (4, 11)
남은 구분: ['오프라인' '온라인' '기업특판(B2B)' '기타(로열티)']
2021년 데이터 추가: (4, 3)
2022년 데이터 추가: (4, 3)
2023년 데이터 추가: (4, 3)
2024년 데이터 추가: (4, 3)
2025년 데이터 추가: (4, 3)

합친 결과:


,channel_kr,sales,year
0,오프라인,"3,259",2021
1,온라인,"1,629",2021
2,기업특판(B2B),"1,702",2021
3,기타(로열티),0,2021
4,오프라인,"7,349",2022
5,온라인,"1,931",2022
6,기업특판(B2B),"2,413",2022
7,기타(로열티),0,2022
8,오프라인,"9,477",2023
9,온라인,"2,229",2023



전체 shape: (20, 3)

sales 변환 후:


...

year
2021    1.0
2022    1.0
2023    1.0
2024    1.0
2025    1.0
Name: ratio, dtype: float64


데이터 타입:
year            int64
channel        object
channel_kr     object
sales           int64
ratio         float64
dtype: object

연도별 총 매출 (백만원):


year
2021     6590
2022    11693
2023    14975
2024    21284
2025    41336
Name: sales, dtype: int64


채널별 총 매출 (백만원):


channel_kr
오프라인         55232
온라인          23198
기업특판(B2B)    16933
기타(로열티)        515
Name: sales, dtype: int64

In [112]:

# ========================================
# Step 21: CSV 저장
# ========================================
df_long.to_csv(r'C:\Users\Comet\Desktop\kpdh\data\processed\museum_shop_sales_processed.csv', index=False, encoding='utf-8-sig')

print("\n" + "="*50)
print("✅ CSV 저장 완료: 판매채널_전처리_완료.csv")
print("="*50)
print(f"저장된 데이터: {len(df_long)}행 × {len(df_long.columns)}열")
print(f"컬럼: {df_long.columns.tolist()}")


✅ CSV 저장 완료: 판매채널_전처리_완료.csv
저장된 데이터: 20행 × 5열
컬럼: ['year', 'channel', 'channel_kr', 'sales', 'ratio']


In [114]:
df_long.dtypes

year            int64
channel        object
channel_kr     object
sales           int64
ratio         float64
dtype: object

In [ ]:
import pandas as pd

# ========================================
# Step 1: 파일 읽기
# ========================================
# 파일 읽기 - 세 번째 시트
df = pd.read_excel(r'C:\Users\Comet\Desktop\kpdh\data\raw\museum\csv\museum_shop_sales.xlsx',sheet_name=1)

# 원본 확인
print("원본 데이터:")
display(df)
print(f"\n원본 shape: {df.shape}")
print(f"\n컬럼: {df.columns.tolist()}")

# ========================================
# Step 2: '계' 행 제거
# ========================================
# 공백 제거 (안전장치)
df['구분'] = df['구분'].astype(str).str.strip()

# '계' 행 확인
print(f"\n'계' 행 개수: {(df['구분'] == '계').sum()}개")

# '계' 행 제거
df = df[df['구분'] != '계'].copy()

print(f"\n'계' 제거 후 shape: {df.shape}")
print(f"남은 구분: {df['구분'].unique()}")

# ========================================
# Step 3-7: 연도별 데이터 처리
# ========================================
# 빈 리스트 생성
dfs = []

# 2021년부터 2025년까지 반복 처리
for year in [2021, 2022, 2023, 2024, 2025]:
    # 현재 연도의 데이터만 추출 (구성비 컬럼 제외)
    temp = df[['구분', f'{year}년']].copy()
    
    # 컬럼명 설정 (한글명 유지)
    temp.columns = ['channel_kr', 'sales']
    
    # 연도 컬럼 추가
    temp['year'] = year
    
    # 리스트에 추가
    dfs.append(temp)
    print(f"{year}년 데이터 추가: {temp.shape}")

# ========================================
# Step 8: 모든 연도 합치기
# ========================================
df_long = pd.concat(dfs, ignore_index=True)

print("\n합친 결과:")
display(df_long.head(10))
print(f"\n전체 shape: {df_long.shape}")

# ========================================
# Step 9: 컬럼 순서 정리
# ========================================
df_long = df_long[['year', 'channel_kr', 'sales']]

# ========================================
# Step 10: sales 데이터 타입 변환
# ========================================
df_long['sales'] = df_long['sales'].astype(str).str.replace(',', '').astype(int)

print("\nsales 변환 후:")
display(df_long.head(5))

# ========================================
# Step 11: ratio 직접 계산
# ========================================
df_long['ratio'] = df_long.groupby('year')['sales'].transform(
    lambda x: (x / x.sum()).round(4)
)

# ========================================
# Step 12: 반올림 오차 보정
# ========================================
for year in df_long['year'].unique():
    year_mask = df_long['year'] == year
    total = df_long.loc[year_mask, 'ratio'].sum()
    diff = 1.00 - total
    
    if diff != 0:
        # 가장 큰 ratio에 조정
        max_idx = df_long[year_mask]['ratio'].idxmax()
        df_long.loc[max_idx, 'ratio'] += diff
        print(f"{year}년 보정: {diff:+.4f}")

# ========================================
# Step 13: 정렬
# ========================================
df_long = df_long.sort_values(['year', 'channel_kr']).reset_index(drop=True)

# ========================================
# Step 14: 단위 컬럼 추가
# ========================================

# ========================================
# Step 15: 채널명 영문 추가
# ========================================
channel_map = {
    '오프라인': 'Offline',
    '온라인': 'Online',
    '기업특판(B2B)': 'B2B',
    '기타(로열티)': 'Royalty'
}

# 영문명 생성 (한글 → 영문)
df_long['channel'] = df_long['channel_kr'].map(channel_map)

# ========================================
# Step 16: 컬럼 순서 정리
# ========================================
df_long = df_long[['year','channel', 'channel_kr', 'sales','ratio']]

# ========================================
# Step 17: 최종 결과 확인
# ========================================
print("\n" + "="*50)
print("최종 결과")
print("="*50)
display(df_long)

# ========================================
# Step 18: 검증
# ========================================
print("\n연도별 ratio 합계:")
ratio_check = df_long.groupby('year')['ratio'].sum()
display(ratio_check)

# ========================================
# Step 19: 데이터 타입 확인
# ========================================
print("\n데이터 타입:")
print(df_long.dtypes)

# ========================================
# Step 20: 기본 통계
# ========================================
print("\n연도별 총 매출 (백만원):")
display(df_long.groupby('year')['sales'].sum())

print("\n채널별 총 매출 (백만원):")
display(df_long.groupby('channel_kr')['sales'].sum().sort_values(ascending=False))

# ========================================
# Step 21: CSV 저장
# ========================================
df_long.to_csv(r'C:\Users\Comet\Desktop\kpdh\data\processed\museum_shop_sales_yearly_2021-2025.csv', index=False, encoding='utf-8-sig')
print("\n" + "="*50)
print("✅ CSV 저장 완료: 판매채널_전처리_완료.csv")
print("="*50)
print(f"저장된 데이터: {len(df_long)}행 × {len(df_long.columns)}열")
print(f"컬럼: {df_long.columns.tolist()}")

원본 데이터:


,구분,2021년,2021년_구성비,2022년,2022년_구성비,2023년,2023년_구성비,2024년,2024년_구성비,2025년,2025년_구성비
0,계,"6,591",100.0%,"11,692",100.0%,"14,976",100.0%,"21,284",100.0%,"41,337",100.0%
1,오프라인,"3,259",49.4%,"7,349",62.9%,"9,477",63.3%,"11,756",55.2%,"23,391",56.6%
2,온라인,"1,629",24.7%,"1,931",16.5%,"2,229",14.9%,"5,222",24.5%,"12,187",29.5%
3,기업특판(B2B),"1,702",25.8%,"2,413",20.6%,"3,171",21.2%,"4,024",18.9%,"5,623",13.6%
4,기타(로열티),0,0.0%,0,0.0%,98,0.7%,282,1.3%,135,0.3%



원본 shape: (5, 11)

컬럼: ['구분', '2021년', '2021년_구성비', '2022년', '2022년_구성비', '2023년', '2023년_구성비', '2024년', '2024년_구성비', '2025년', '2025년_구성비']

'계' 행 개수: 1개

'계' 제거 후 shape: (4, 11)
남은 구분: ['오프라인' '온라인' '기업특판(B2B)' '기타(로열티)']
2021년 데이터 추가: (4, 3)
2022년 데이터 추가: (4, 3)
2023년 데이터 추가: (4, 3)
2024년 데이터 추가: (4, 3)
2025년 데이터 추가: (4, 3)

합친 결과:


,channel_kr,sales,year
0,오프라인,"3,259",2021
1,온라인,"1,629",2021
2,기업특판(B2B),"1,702",2021
3,기타(로열티),0,2021
4,오프라인,"7,349",2022
5,온라인,"1,931",2022
6,기업특판(B2B),"2,413",2022
7,기타(로열티),0,2022
8,오프라인,"9,477",2023
9,온라인,"2,229",2023



전체 shape: (20, 3)

sales 변환 후:


...


데이터 타입:
year            int64
channel        object
channel_kr     object
sales           int64
ratio         float64
dtype: object

연도별 총 매출 (백만원):


year
2021     6590
2022    11693
2023    14975
2024    21284
2025    41336
Name: sales, dtype: int64


채널별 총 매출 (백만원):


channel_kr
오프라인         55232
온라인          23198
기업특판(B2B)    16933
기타(로열티)        515
Name: sales, dtype: int64


✅ CSV 저장 완료: 판매채널_전처리_완료.csv
저장된 데이터: 20행 × 5열
컬럼: ['year', 'channel', 'channel_kr', 'sales', 'ratio']


In [121]:
import pandas as pd

# ========================================
# Step 1: 파일 읽기
# ========================================
df = pd.read_excel(r'C:\Users\Comet\Desktop\kpdh\data\raw\museum\csv\museum_shop_sales.xlsx',sheet_name=0)

# 원본 확인
print("원본 데이터:")
display(df)
print(f"\n원본 shape: {df.shape}")

# ========================================
# Step 2: '계' 행 제거
# ========================================
df = df[df['구분'] != '계'].copy()

print("\n'계' 제거 후:")
display(df)
print(f"제거 후 shape: {df.shape}")

# ========================================
# Step 3: Wide → Long 변환
# ========================================
df_long = pd.melt(
    df,
    id_vars=['구분'],
    value_vars=['오프라인', '온라인'],
    var_name='channel_kr',
    value_name='sales'
)

print("\nLong 형태 변환:")
display(df_long.head(10))

# ========================================
# Step 4: 월 숫자 추출
# ========================================
df_long['month'] = df_long['구분'].str.replace('월', '').astype(int)

# ========================================
# Step 5: 연도 추가
# ========================================
df_long['year'] = 2025

# ========================================
# Step 6: date 컬럼 생성 (year + month)
# ========================================
df_long['date'] = pd.to_datetime(
    df_long['year'].astype(str) + '-' + 
    df_long['month'].astype(str).str.zfill(2) + '-01'
)

print("\ndate 컬럼 생성:")
display(df_long.head(10))

# ========================================
# Step 7: sales 데이터 변환
# ========================================
df_long['sales'] = df_long['sales'].astype(str).str.strip().str.replace(',', '').astype(int)

# ========================================
# Step 8: 채널명 영문 추가 (첫글자 대문자)
# ========================================
channel_map = {
    '오프라인': 'Offline',
    '온라인': 'Online'
}
df_long['channel'] = df_long['channel_kr'].map(channel_map)

# ========================================
# Step 9: 컬럼 순서 정리 (date 추가)
# ========================================
df_long = df_long[['date', 'year', 'month', 'channel', 'channel_kr', 'sales']]

# ========================================
# Step 10: 정렬
# ========================================
df_long = df_long.sort_values(['date', 'channel_kr']).reset_index(drop=True)

# ========================================
# 최종 결과
# ========================================
print("\n" + "="*50)
print("최종 결과")
print("="*50)
display(df_long)

print(f"\n최종 shape: {df_long.shape}")

# ========================================
# 검증
# ========================================
print("\n채널별 총 매출:")
display(df_long.groupby('channel_kr')['sales'].sum())

print("\n월별 총 매출:")
display(df_long.groupby('month')['sales'].sum())

print("\n데이터 타입:")
print(df_long.dtypes)

# ========================================
# CSV 저장
# ========================================
df_long.to_csv(r'C:\Users\Comet\Desktop\kpdh\data\processed\museum_shop_sales_monthly_2025.csv', index=False, encoding='utf-8-sig')

print("\n" + "="*50)
print("✅ CSV 저장 완료: museum_shop_sales_monthly_2025.csv")
print("="*50)
print(f"저장된 데이터: {len(df_long)}행 × {len(df_long.columns)}열")
print(f"컬럼: {df_long.columns.tolist()}")
print(df_long.dtypes)


원본 데이터:


,구분,오프라인,온라인
0,1월,909,435
1,2월,999,631
2,3월,953,601
3,4월,"1,068",653
4,5월,"1,343",568
5,6월,"1,349",562
6,7월,"2,503","2,037"
7,8월,"3,428","1,370"
8,9월,"2,539","1,233"
9,10월,"3,123","1,346"



원본 shape: (13, 3)

'계' 제거 후:


,구분,오프라인,온라인
0,1월,909,435
1,2월,999,631
2,3월,953,601
3,4월,"1,068",653
4,5월,"1,343",568
5,6월,"1,349",562
6,7월,"2,503","2,037"
7,8월,"3,428","1,370"
8,9월,"2,539","1,233"
9,10월,"3,123","1,346"


제거 후 shape: (12, 3)

Long 형태 변환:


...


최종 shape: (24, 6)

채널별 총 매출:


channel_kr
오프라인    23391
온라인     12187
Name: sales, dtype: int64


월별 총 매출:


month
1     1344
2     1630
3     1554
4     1721
5     1911
6     1911
7     4540
8     4798
9     3772
10    4469
11    3779
12    4149
Name: sales, dtype: int64


데이터 타입:
date          datetime64[ns]
year                   int64
month                  int64
channel               object
channel_kr            object
sales                  int64
dtype: object

✅ CSV 저장 완료: museum_shop_sales_monthly_2025.csv
저장된 데이터: 24행 × 6열
컬럼: ['date', 'year', 'month', 'channel', 'channel_kr', 'sales']
date          datetime64[ns]
year                   int64
month                  int64
channel               object
channel_kr            object
sales                  int64
dtype: object


In [ ]:
# CSV 저장
df_long.to_csv(r'C:\Users\Comet\Desktop\kpdh\data\processed\museum_shop_sales_monthly_2025.csv', index=False, encoding='utf-8-sig')
print("\n" + "="*50)
print("✅ CSV 저장 완료: 판매채널_전처리_완료.csv")
print("="*50)
print(f"저장된 데이터: {len(df_long)}행 × {len(df_long.columns)}열")
print(f"컬럼: {df_long.columns.tolist()}")